In [13]:
import os
import math
import numpy as np
import pickle
import torch
from utils import load_model
from problems.tsp.problem_tsp import TSPDataset
import time
from mcts.mcts_utils import evaluate_tour
from utils.beam_search import beam_search
from eval import get_best

In [29]:
### PARAMETERS TO BE SET ###
n_graphs = 100
n_nodes = 20
width = 3
temperature = 1

In [15]:
# Load model 20 nodes and set temperature
model, _ = load_model(F'pretrained/tsp_{n_nodes}/')
model.eval()
print("model loaded")
# Load test set graphs 20 nodes
# If this block does not work, make sure you called:
# python generate_data.py --problem all --name test --seed 1234
with open(F"data/tsp/tsp{n_nodes}_test_seed1234.pkl", "rb") as f:
    data = pickle.load(f)
    dataset = TSPDataset(None, 0, 0, 0, None)
    dataset.data = [torch.FloatTensor(row) for row in (data[0:0+10000])]
    dataset.size = len(dataset.data)
    graphs = []
    for sample in dataset.data:
        graphs.append(sample)

  [*] Loading model from pretrained/tsp_20/epoch-99.pt
model loaded


In [ ]:
# Greedy experiments
greedy_timestamps = []
greedy_results = []

for i in range(n_graphs):
    graph = graphs[i]
    graph_batch = graph[None] # Add batch dimension
    tour = [0] # Start at first node, unconventional, TODO: fix this
    t_s = time.perf_counter()
    with torch.no_grad():
        embeddings, _ = model.embedder(model._init_embed(graph_batch))

        # Compute keys, values for the glimpse and keys for the logits once as they can be reused in every step
        fixed = model._precompute(embeddings)
        for visit in range(graph.shape[0] - 1):
            tour_tensor = torch.tensor(tour).long()
            if len(tour_tensor) == 0:
                step_context = model.W_placeholder
            else:
                step_context = torch.cat((embeddings[0, tour_tensor[0]],
                                        embeddings[0, tour_tensor[-1]]), -1)
            query = fixed.context_node_projected + model.project_step_context(step_context[None, None, :])
            mask = torch.zeros(graph_batch.shape[1], dtype=torch.uint8) > 0
            mask[tour_tensor] = 1
            mask = mask[None, None, :]

            log_p, _ = model._one_to_many_logits(query, fixed.glimpse_key, fixed.glimpse_val, fixed.logit_key, mask)
            p = torch.softmax(log_p / temperature, -1)[0, 0]
            assert (p[tour_tensor] == 0).all()
            assert (p.sum() - 1).abs() < 1e-5
            p = p.numpy()
            next_node = np.argmax(p)
            tour.append(next_node)
        t_e = time.perf_counter()
        tour.append(0) # Return to the starting position
        tour_len = evaluate_tour(graph.numpy(), tour)
        greedy_results.append(tour_len)
        greedy_timestamps.append(t_e - t_s)
        print("\nGraph", i)
        print(evaluate_tour(graph.numpy(), tour))

In [5]:
print("Average greedy results", np.mean(greedy_results))
print("Average duration", np.mean(greedy_timestamps))

Average greedy results 4.9277196
Average duration 0.0100973587999988


In [15]:
greedy_results_dict = {
    "greedy_result" : greedy_results,
    "greedy_timestamps" : greedy_timestamps
}

In [17]:
save_string = F"experiments/benmark_greedy.pkl"
with open(save_string, "wb") as f:
    pickle.dump(greedy_results_dict, f)

In [37]:
# Beam search experiments
bs_timestamps = []
bs_results = []

for i in range(n_graphs):
    graph = graphs[i]
    graph_batch = graph[None] # Add batch dimension
    t_s = time.perf_counter()
    with torch.no_grad():
        cum_log_p, sequences, costs, ids, batch_size = model.beam_search(graph_batch, beam_size=width)
        t_e = time.perf_counter()
        if sequences is None:
            sequences = None
            costs = math.inf
        else:
            sequences, costs = get_best(
                sequences.cpu().numpy(), costs.cpu().numpy(),
                ids.cpu().numpy() if ids is not None else None,
                batch_size
            )
        tour = sequences[0].tolist()
        tour.append(tour[0]) # Return to the starting position
        tour_len = evaluate_tour(graph.numpy(), tour)
        bs_results.append(tour_len)
        bs_timestamps.append(t_e - t_s)
        print("\nGraph", i)
        print(tour_len)
        print(tour)

/Users/gerlise/Documents/GitHub/attention-learn-to-route/nets/attention_model.py:177: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  flat_parent = torch.arange(flat_action.size(-1), out=flat_action.new()) // ind_topk.size(-1)



Graph 0
3.649399
[16, 2, 15, 12, 4, 9, 17, 6, 1, 13, 19, 18, 3, 10, 7, 5, 14, 0, 11, 8, 16]

Graph 1
3.7619956
[6, 16, 11, 4, 10, 18, 2, 15, 9, 8, 7, 13, 17, 1, 5, 3, 19, 0, 14, 12, 6]

Graph 2
3.7796686
[12, 17, 4, 11, 1, 19, 10, 3, 8, 6, 16, 14, 9, 13, 5, 0, 7, 15, 18, 2, 12]

Graph 3
4.118915
[8, 6, 11, 15, 18, 0, 17, 1, 19, 9, 5, 3, 10, 12, 4, 7, 16, 2, 14, 13, 8]

Graph 4
3.2645254
[15, 14, 5, 3, 10, 9, 11, 2, 1, 4, 19, 6, 8, 13, 7, 16, 0, 17, 12, 18, 15]

Graph 5
3.8607295
[7, 19, 2, 0, 11, 16, 1, 9, 18, 6, 12, 5, 15, 3, 14, 4, 10, 13, 17, 8, 7]

Graph 6
3.8685248
[13, 14, 18, 12, 17, 10, 7, 9, 15, 5, 11, 8, 4, 3, 1, 19, 2, 0, 16, 6, 13]

Graph 7
3.9224017
[4, 14, 19, 3, 6, 8, 18, 10, 5, 13, 0, 16, 17, 11, 2, 7, 12, 1, 9, 15, 4]

Graph 8
4.1289177
[2, 19, 8, 12, 9, 0, 10, 18, 15, 6, 13, 17, 5, 7, 4, 14, 16, 1, 3, 11, 2]

Graph 9
4.0068173
[15, 5, 14, 2, 17, 8, 11, 18, 3, 13, 16, 6, 12, 19, 1, 7, 10, 9, 0, 4, 15]

Graph 10
3.7787595
[16, 9, 11, 19, 12, 8, 6, 0, 4, 10, 2, 14, 5, 7

In [36]:
print("Average beam search results", np.mean(bs_results))
print("Average duration", np.mean(bs_timestamps))

Average beam search results 3.8423274
Average duration 0.01734086385995852


In [18]:
bs_results_dict = {
    "bs_result" : bs_results,
    "bs_timestamps" : bs_timestamps
}

In [19]:
save_string = F"experiments/beam_search_{width}.pkl"
with open(save_string, "wb") as f:
    pickle.dump(bs_results_dict, f)